<a href="https://colab.research.google.com/github/devilteo911/gigi-d-alessio-lyrics-generator/blob/dev/DistilGPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Apr  3 15:45:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 6.5 MB 32.9 MB/s 
     |████████████████████████████████| 596 kB 40.0 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 895 kB 43.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import os
import time
import datetime
import re
import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt

import torch
from torch.utils.data import (
    Dataset,
    DataLoader,
    random_split,
    RandomSampler,
    SequentialSampler,
)

torch.manual_seed(42)

from transformers import AutoTokenizer, AutoModel
from transformers import AdamW, get_linear_schedule_with_warmup
from pathlib import Path

In [ ]:
path = str(Path().cwd()) + "/dataset/"
list_of_contents = []
for file_ in Path(path).glob("*.csv"):
    file_content = ""
    with open(file_, "r") as f:
        file_content = " ".join(f.readlines()).strip()
    file_content = re.sub("\n|\t|\|:||-|[|?|]|!|,|\"", " ", file_content)
    # file_content = re.sub("\'", "", file_content)
    file_content = re.sub("(|)", "", file_content)
    # weird vocals
    file_content = re.sub("â", "a", file_content)
    file_content = re.sub("ê", "e", file_content)
    file_content = re.sub("â", "a", file_content)
    file_content = re.sub("ô", "o", file_content)
    file_content = re.sub("ç", "c", file_content)

    list_of_contents.append(file_content)
df = pd.DataFrame(list_of_contents, columns=["content"])

# %%
texts = df.content.copy()  # just use the main bio text in this example
texts  # %%

0     Je sto vicino a te cu ciento strilla attuorno ...
1     Viento scioscia stanottetrase pe' sotta e fatt...
2     E te sento quanno scinne 'e scale  'e corza se...
3     Je so' pazzo  je so' pazzo e vogl'essere chi v...
4     Comm'è triste e comm'è amaro  st'assettato e g...
5     Comm'è triste e comm'è amaro  st'assettato e g...
6     Napule è mille culure  Napule è mille paure  N...
7     E sotto 'o sole vene e se ne va' e saglie sula...
8     Tu diciv' nun da retta  nun ce penzà  da 'na m...
9     Comm'è triste  comm'è amaro Assettarse pe guar...
10    My name your name Ma che importanza ha E tu gu...
11    Passa 'o tiempo e che fa  tutto cresce e se ne...
12    Pecchè  pecchè  pecchè nun truov' pace   E chi...
13    A me me piace 'o blues e tutt'e journe aggio c...
14    Saglie  saglie Cu' sta spòrta chièna d'aglie S...
15    Nuje ca cercammo Dio stammo pè sempe annure nu...
16    Napule è mille culure  Napule è mille paure  N...
17    E te sento quanno scinne 'e scale  'e corz

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("GroNLP/gpt2-small-italian-embeddings")


Downloading:   0%|          | 0.00/135 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/970 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/475k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/280k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [ ]:
train_set = texts[:int(len(texts)*0.8)]
test_set = texts[int(len(texts)*0.8):]

train_dataset, test_dataset = "", ""

for song in train_set:
    train_dataset += song + " "

for song in test_set:
    test_dataset += song + " " 

In [ ]:
with open("train_dataset.txt", "w") as f:
    f.write(train_dataset)

with open("test_dataset.txt", "w") as f:
    f.write(test_dataset)

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

train_path = '/content/train_dataset.txt'
test_path = '/content/test_dataset.txt'

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("GroNLP/gpt2-small-italian-embeddings")


training_args = TrainingArguments(
    output_dir="./gptAlessio", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=100, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    evaluation_strategy = "epoch",
    learning_rate=1.372e-5,
    weight_decay=0.01,
    do_eval=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:882: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/427M [00:00<?, ?B/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 80
  Num Epochs = 100
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 300


Epoch,Training Loss,Validation Loss
1,No log,5.381484
2,No log,5.378283
3,No log,5.372711
4,No log,5.364937
5,No log,5.355157
6,No log,5.343674
7,No log,5.330719
8,No log,5.316536
9,No log,5.301371
10,No log,5.285657


***** Running Evaluation *****
  Num examples = 20
  Batch size = 64
***** Running Evaluation *****
  Num examples = 20
  Batch size = 64
***** Running Evaluation *****
  Num examples = 20
  Batch size = 64
***** Running Evaluation *****
  Num examples = 20
  Batch size = 64
***** Running Evaluation *****
  Num examples = 20
  Batch size = 64
***** Running Evaluation *****
  Num examples = 20
  Batch size = 64
***** Running Evaluation *****
  Num examples = 20
  Batch size = 64
***** Running Evaluation *****
  Num examples = 20
  Batch size = 64
***** Running Evaluation *****
  Num examples = 20
  Batch size = 64
***** Running Evaluation *****
  Num examples = 20
  Batch size = 64
***** Running Evaluation *****
  Num examples = 20
  Batch size = 64
***** Running Evaluation *****
  Num examples = 20
  Batch size = 64
***** Running Evaluation *****
  Num examples = 20
  Batch size = 64
***** Running Evaluation *****
  Num examples = 20
  Batch size = 64
***** Running Evaluation *****
  N

TrainOutput(global_step=300, training_loss=4.359129638671875, metrics={'train_runtime': 504.0493, 'train_samples_per_second': 15.871, 'train_steps_per_second': 0.595, 'total_flos': 522584064000000.0, 'train_loss': 4.359129638671875, 'epoch': 100.0})

In [ ]:
trainer.save_model()

Saving model checkpoint to ./gptAlessio
Configuration saved in ./gptAlessio/config.json
Model weights saved in ./gptAlessio/pytorch_model.bin


In [ ]:
from transformers import pipeline

# http://www.ndonio.it/Napoli%20Glossario.htm dizionario napoletano da aggiungere ai token

gptAlessio = pipeline('text-generation',model='./gptAlessio', tokenizer="GroNLP/gpt2-small-italian-embeddings")#,config={'max_length':800})

loading configuration file ./gptAlessio/config.json
Model config GPT2Config {
  "_name_or_path": "./gptAlessio",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 0,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 100,
      "no_repeat_ngram_size": 4,
      "num_beams": 10,
      "repetition_penal

In [ ]:
#@title Install dependencies

%%capture
!pip install transformers
!pip install datasets
!pip install torch
!pip install wandb
!pip install lyricsgenius
!pip install aiohttp
!pip install langdetect
!pip install accelerate
!pip install --upgrade jax jaxlib 
!pip install --upgrade git+https://github.com/google/flax.git
!pip install tqdm --upgrade
!pip install hf-lfs
!git lfs install


In [ ]:
from IPython.display import display, HTML, Javascript, clear_output
import lyricsgenius
from tqdm.notebook import tqdm as bar
import requests
from bs4 import BeautifulSoup
import re
from datasets import Dataset, DatasetDict
import numpy as np
import time
import os
import json
import langdetect
import datetime
from pathlib import Path
import wandb
import pathlib
import nest_asyncio
nest_asyncio.apply()

import asyncio
from concurrent.futures import ProcessPoolExecutor
import sys
sys.setrecursionlimit(99999)
import aiohttp


In [ ]:
def stylize():
    "Handle dark mode"
    display(HTML('''
    <style>
    :root {
        --table_bg: #EBF8FF;
    }
    html[theme=dark] {
        --colab-primary-text-color: #d5d5d5;
        --table_bg: #2A4365;
    }
    .jupyter-widgets {
        color: var(--colab-primary-text-color);
    }
    table {
        border-collapse: collapse !important;
    }
    td {
        text-align:left !important;
        border: solid var(--table_bg) !important;
        border-width: 1px 0 !important;
        padding: 6px !important;
    }
    tr:nth-child(even) {
        background-color: var(--table_bg) !important;
    }
    .table_odd {
        background-color: var(--table_bg) !important;
        margin: 0 !important;
    }
    .table_even {
        border: solid var(--table_bg) !important;
        border-width: 1px 0 !important;
        margin: 0 !important;
    }
    .jupyter-widgets {
        margin: 6px;
    }
    .widget-html-content {
        font-size: var(--colab-chrome-font-size) !important;
        line-height: 1.24 !important;
    }
    </style>'''))

def post_process(output_sequences):
    predictions = []
    generated_sequences = []

    max_repeat = 2

    # decode prediction
    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        generated_sequence = generated_sequence.tolist()
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)
        generated_sequences.append(text.strip())

    #return generated_sequences               
    for i, g in enumerate(generated_sequences):
        res = str(g).replace('\n\n\n', '\n').replace('\n\n', '\n')
        lines = res.split('\n')
        # print(lines)
        # i = max_repeat
        # print(i)
        # while i != len(lines):
        #     remove_count = 0
        #     for index in range(0, max_repeat):
        #         print(i - index - 1, i - index)
        #         if lines[i - index - 1] == lines[i - index]:
        #             remove_count += 1
        #     if remove_count == max_repeat:
        #         lines.pop(i)
        #         i -= 1
        #     else:
        #         i += 1
        predictions.append('\n'.join(lines))

    return predictions

def get_table(table_data):
  html = ("</head>\r\n"
    "<body>\r\n\r\n"
    "<h2></h2>"
    "\r\n\r\n"
    "<table>\r\n"
    "    <colgroup>\r\n"
    "       <col span=\"1"
    "\" style=\"width: 10"
    "%;\">\r\n"
    "       <col span=\"1"
    "\" style=\"width: 10"
    "0%;\">\r\n"
    "    </colgroup>\r\n"
    f"{' '.join(table_data)}"
    "</table>\r\n\r\n"
    "</body>\r\n"
    "</html>")
  
  return html

In [ ]:
# @title Generate
#@markdown Enter starting sentence:
start = "Napule" #@param {type:"string"}
#@markdown Amount of generated texts:
num_sequences =   1#@param {type:"integer"}
#@markdown Generation settings:
min_length =  150 #@param {type:"integer"}
max_length =   300#@param {type:"integer"}
temperature = 1 #@param {type:"slider", min:0, max:3, step:0.01}
top_p = 0.95 #@param {type:"slider", min:0, max:1, step:0.01}
top_k = 50 #@param {type:"integer"}
repetition_penalty =  10.0#@param {type:"number"}

encoded_prompt = tokenizer(start, add_special_tokens=False, return_tensors="pt").input_ids
encoded_prompt = encoded_prompt.to(trainer.model.device)
# prediction
output_sequences = trainer.model.generate(
                        input_ids=encoded_prompt,
                        max_length=max_length,
                        min_length=min_length,
                        temperature=float(temperature),
                        top_p=float(top_p),
                        top_k=int(top_k),
                        do_sample=True,
                        repetition_penalty=repetition_penalty,
                        num_return_sequences=num_sequences
                        )


In [ ]:
predictions = post_process(output_sequences)

In [ ]:
table_data = []
for result in predictions:
    table_data.append('<tr><td>' + result.replace("\n", "<br>") + '</td></tr>')
display(HTML(get_table(table_data)))

"Napule e l'acqua è in mare. E chi fa che Naope nun vede niente se ne va: mi piace sventola con un pezzettino sudore...che dice tutt 'e cagna mammaè volato ad stimmata'. Ma tu so senza guardarsi lampo! Se vuoi a scurervi, ti spadaci per via cchiù"". Non era sposato ma da tredici al nove vendette pane fin dalla giovinezza.. Si tenebravi anche le chiavi di casa poi annetterestionaglie istattiere cornecchie dove fanno casino tutte quelle scale"" Oggi forse non hai voglia perché quando sono nato? (Indrosa de mo culure) Nunànno me nadde quanno cu te scartraggio si fermì fà sacca comenza chiove stuerò chello feellusca dulture mracconna mumento ‘steacciosamente dispornu ccca rimperia nassuta allerta.. Chi pussi passarti o lascia presagità sempre più scenne A scuola cammriniva restammo croce trëte terra canٍ parodonose traggetuate rícido vuetto toccette nu sole poro scilata junto je capى prono stonato tutto tondo male accussére criceva gnente semprì comme verghe vita ci guardammusivano poco cocesso pure scovento h"
